# Simple MAST-1D Tutorial Run: Single Reach, Constant Parameters

This run sets up the basic input parameters for a MAST-1D run as an object, then passes these to the model. The run here is simple, with no lateral sediment supply and with a single, constant slope, constant parameter reach.

To use this notebook, it should be saved in the MAST_1D folder of the MAST-1D project, as cloned from GitHub: https://github.com/jwlauer/MAST-1D-2.0. 

## Import required libraries

Import the required standard libraries that will be used for setting up the inputs. There are a number of other libraries importated by the MAST-1D classes. An Anaconda distribution of Python should include the required libraries.

In [17]:
import os
import sys
import csv
sys.path.append("..")
from Hydrology.clsTimeSeries import clsTimeSeries
from MAST_1D.clsModel import clsModel
from MAST_1D.clsInputs import clsInputs

Create an empty instance of the inputs object that will be used to store a wide range of input data and then pass it to the model.

In [18]:
inputs = clsInputs() 

## I.A  Import csv with node-specific inputs for some attributes (optional)

Here you can import a .csv file with spacially-explicit initial conditions. It is commented out in this example and has not been tested with recent revisions to the model.  However, in theory, it is straightforward to create a spreadsheet and/or input textfile that is used to specify basic basic properties for each node (width, reach length represented, coordinate, elevation, etc.).

In [56]:
#inputfile = os.path.join(os.pardir, 'Elwha_spatial_data', 'UR_Valley_Nodes_sinuosity.csv')
#
## open the file in universal line ending mode 
#with open(inputfile, 'rU') as infile: # This loop copied from Stack Overflow.
#  # read the file as a dictionary for each row ({header : value})
#  reader = csv.DictReader(infile)
#  data = {}
#  for row in reader:
#    for header, value in row.items():
#      try:
#        data[header].append(value)
#      except KeyError:
#        data[header] = [value]
#        
#inputs.BcNodes = map(lambda x: float(x), data['InitialBc'])
#inputs.BfNodes = map(lambda x: float(x), data['Avg_width'])
#inputs.dxf = map(lambda x: float(x), data['Valley_length']) # Optional length of valley to determine node length with sinuosity
#inputs.Canyon = map(lambda x: bool(int(x)), data['Canyon'])
#inputs.ReachwideBedrock = False # If true, every reach will only be allowed to degrade a user-specified amount and bed will become 'partly-alluvial'
#inputs.PartlyAlluvialMin = -.2
#inputs.ChSinNodes = map(lambda x: float(x), data['Avg_Sinuosity'])

## I.B  Import reach conditions from a previous run (optional)

Similar to the code commented above, it is possible to save a run and then simply re-load it as the intial condition for a future run. Again, this is commented out here and is relatively untested.  However, the code later looks for the inputs.initialcond flag, so this must be set either to "True" or "False".

In [57]:
inputs.initialcond = False # If you are using a prior run as initial conditions, True; if starting from scratch, False
#inputs.priorReach = 'D:\MAST-1D_version_K6\Output\Pre_vs_post_dam\WholePeriod'+ '//' +'save.Reach' # File with initial conditions Reach object

## II. Set Reach-Average Channel Geometry

Set reach-average channel geometry. Note that you must specify these variables even if you previously loaded an initial conditions file from a previous run or defined nodes on a case-by-case basis. MAST-1D will set the feed and floodplain number based on the reach-average values specified here.

In [37]:
inputs.Nnodes = 66 #  Number of nodes
inputs.Bc = 94. # Channel width (m)
inputs.reachlength = 13673. # Length of reach (channel length) (m)
inputs.Bf = 500 # Total valley width (m)
inputs.ChSin = 1.02 # Channel sinuosity
inputs.Slope = 0.0074 # Bed gradient

## III. Choose Hydrograph or Flow Duration Curve Run

The model allows for hydrographs or flow duration curves. For long-term runs, hydrograph data may not be available to represent the entire period of simualtion. In these cases, set the cyclinghydrograph flag to true, and the hydrograph will repeat.  In this section, the parameters for flow duration curves are also specified.

In [39]:
inputs.Hydrograph = True # True if supplying hydrograph instead of flow duration curve--not used for this demo
inputs.CyclingHydrograph = False #Flag to determine if hydrograph is repeated

## IV.A Timestep parameters for flow duration curve

The total run length is also specified here using the MaxSteps variable.

In [40]:
inputs.MaxSteps = 2000 #  Total number of timesteps to run.
inputs.dt = [0] # Timestep (years)--can add multiple values for timestep adjustment during run
inputs.dtcount = [] # List of timesteps to instigate timestep interval change (length of list should be one fewer than dt)

## IV.B  Timestep parameters for hydrograph
MAST-1D is set up to run hydrographs with a daily resolution.  It will keep track
of the date and instigate user-specified boundary condition changes based on the date.

In [ ]:
inputs.Tmult = 1 # Number of timesteps per day
inputs.TmultHighFlow = 8 #150 # Number of timesteps for days with flow above High Flow Timestep Threshold
inputs.TmultCyclingHydrograph = 27 #Number of timesteps per day for Cycling Hydrograph runs.
inputs.StartDate = (2011,9,15) # (year, month, day).
inputs.LowFlowThreshold = 40 # threshold discharge below which hydrograph runs ignore sediment transport
inputs.HighFlowTimestepThreshold = 120 #300 # threshold discharge above which hydrograph runs use a shorter timestep.

## V. Specify Boundary Conditions

The first of these is specific to a dam removal run. If the removal flag is set to true, the model looks for this flag and can use it to change boundary conditions at times that are hard-coded in the model. This should normally be false.

In [21]:
inputs.Removal = False # This is a custom variable that is used to set some boundary
    # condition behavior for the Elwha River Dam removal (see Section VI.D in 
    # clsModel).

### Sediment
The upstream feed is determined as a fraction of the computed sediment transport capacity of the upstream node. LoadFactor is a list that specifies what fraction of that capacity is provided as sediment feed at a given point in the run. The same factor applies to all sediment size classes.  

The sediment supply can change at a user specified timestep, LoadFactorCount. This should be provided as a list if timesteps where a change in feed occurs (if using a duration-based run) or a tuple ((yy,m,dd)) for hydrograph runs.  Number of entries should be one less than LoadFactor.

Feed type should be 'DurationCurve' if using a duration curve or 'RatingCurve' if using a hydrograph.  You can also create a custom method for applying feed--see Section VI.E in clsModel. Note that for many runs, supply is specified at the capacity of the upstream node.

In [22]:
inputs.LoadFactor = [0.8,0.8]  
inputs.LoadFactorCount = [900] 
inputs.FeedType = "RatingCurveUpperElwha" 

### Hydraulics
The downstream boundary for the gradually varied flow computation can be either a specified water surface or normal depth.  SetBoundary should be False for normal depth.  If a specified water surface elevation is used, it is set in BoundaryFactor.  The water surface elevation can change at timesteps specified in BoundaryFactorCount, which is a list of times to instigate WSE change.  The number of entries shold be one less than BoundaryFactor.

In [23]:
inputs.SetBoundary = False 
inputs.BoundaryFactor = [30.]
inputs.BoundaryFactorCount = [] 

## VI: Hydraulic Control Parameters

The backwater computation can be performed using either a Chezy equation or a Manning Equation to represent friction losses.  Set vfunc to True for Manning.  Friction factors for channel and floodplain, as well as form roughness and sinuosity factors and floodplain mannings roughness are also specified.

In [24]:
inputs.ManningStabilizer = 3 # Controls how much the water surface elevation changes
    #with each iteration in the backwater calculation
inputs.vfunc = True # Velocity function, True for Manning, False for Chezy
inputs.Cfc = 0.0075 # 1/Cz^2 for channel
inputs.Cff = 0.69 # 1/Cz^2 for floodplain
inputs.ncAddons = 0.0066 # Form roughness component for Manning's n; 
    # will be added to calculated n value
inputs.ncMultiplier = 1.15 # Sinuosity multiplier for Manning's n
inputs.nf = 0.1 # Manning's n for floodplain

## VII.A Discharge Record for Flow Duration Curve
Discharges are read into a list describing the characteristic discharge for each bin of the flow duration distribution.  Probabilities p are specified for each bin. These represent the fraction of time a given discharge occurs.

In [25]:
inputs.Qw = [\
30.,\
50.,\
70.,\
90.,\
150.,\
275.,\
455.,\
540.] 

inputs.p = [\
0.3,\
0.5,\
0.1,\
0.05,\
0.045,\
0.004,\
0.0007,\
0.0003]

## VII.B Discharge Record for Hydrograph
Discharge timeseries can be specified on a node-by-node basis. Each node is given a code that corresponds with the discharge files specified here.  An along-channel coordinate representing the downstream end of the region of application of a given discharge file can also be specified.

Discharge files should be stored in the "Discharge_Files" folder in the parent directory. See examples there for formatting.  

MAST-1D also calculates initial floodplain and feed parameters based on a flow duration curve representing the modeled period, even when the hydrograph run is turned on.  A flow duration curve will be calculated automatically using DischargeFile and the NumberOfHydroBins variable. 

In [26]:
inputs.DischargeFileID = [0]
inputs.DischargeFileCoords = [0] 
inputs.DischargeFiles = ['Qfile15Sep2011pres']
inputs.NumberofHydroBins = 15

## VIII. Grain Size and Sediment Related Parameters

Set the bin boundaries (mm) for grain size distributions. The finest size class should be the silt/clay. There must be a lower bound (here it is estimated to be 0.002).

In [27]:
inputs.Dbdy = [\
0.002,\
.063,\
1.0,\
2.0,\
4.0,\
8.0,\
16.0,\
32.0,\
64.0,\
128.0,\
256.0,\
512.0,\
1024.]

Set up the initial grain size distribution for the active layer of the channel. The length of the list should be one less than the number of grain size bin boundaries. In MAST-1D, the symbol "F" always represents a grain size fraction, so Fa indicates size fractions in the active layer. If these do not add up to 1, MAST-1D will normalize.

In [28]:
inputs.Fa = [
0.,\
7.0,\
4.3,\
4.7,\
4.7,\
5.7,\
7.4,\
15.,\
21.,\
20.,\
9.7,\
1.]

Set up a possible sediment lag depsit in the substrate. To add lag deposits to the substrate, choose a fraction for the grainsize of choice.  If the lists are left blank, the substrate is composed of the same material as the Active Layer and Floodplain.

In [29]:
inputs.DLag = [] # List of indexes of grainsizes to alter
inputs.FLag = [] # Fraction of GSD

Set other sediment-related parameters. In the past, there was a variable for computing sand load in suspension as a fraction of the sand load computed using the bed material transport formula. This was intended to result in realistic sand deposition rates on the floodplain, but this function is currently not enabled.  The MudFraction variable is similar in that the supply of material FINER than sand is specified as a fraction of the transport rate of the finest bed material size fraction.

The floodplain number for bed material represents the fraction of bed material flux in water flowing across the floodplain that would get stored in the floodplain.

It is possible to specify nodes where lateral sediment supply is provided.  This is done using a list of the nodes where lateral supply is provided, and then a list of multipliers (one for each node). The multiplier represents the fraction of the load computed for the upstream node that is supplied at the respective lateral source node.  To try this out, use the commented lines.

In [30]:
# Parameters for suspended load
inputs.FSandSusp = 0.12 # Fraction of sand in the suspension.  This parameter is 
    # not currently connected in the model.
inputs.MudFraction = 18. # Mud feed multiplier (multiple of next finest size class)
inputs.FlBed = .75 # Floodplain number for bed material

# Bedload sediment transport equation
inputs.TransFunc = 'WilcockCrowe' # Transport function; choices are 'WrightParker' and 'WilcockCrowe'
inputs.TrinityFit = True # True is Trinity River form (Gaeuman 2009), False is normal Wilcock and Crowe        
inputs.CalibrationFactor = 1.  # Calibration coefficient for critical shear stress

# Lateral Sediment Supply
#inputs.LateralSedimentSourceNodes = [20,60]  # indices of nodes with lateral supply
#inputs.LateralMultiplier = [0.1,0.1]  # fraction of upstream supply for lateral source nodes

inputs.LateralSedimentSourceNodes = []  # indices of nodes with lateral supply
inputs.LateralMultiplier = []  # fraction of upstream supply for lateral source nodes


## VIII. Channel migration and width change

In this section, there are flags that control how channel migration is handled in the model. If the widthchange flag is turned on, the constant migration rate is not used, and migration is computed as a function of bank erosion rate and vegetation encroachment rates.  If it is off, the lateral migration is a specified constant.  However, the constant migration rate has to be specified regardless of run type so that a plausible floodplain number can be computed that results in reasonable steady-state bank heights.

In [54]:
inputs.WidthChange = True # If true, turns off constant migration rate and 
    # channel-floodplain coupling is determined by width change functions
inputs.migration = 1.2 # Channel migration rate (m/yr).  

### Vegetation Encroachment Terms

In [31]:
inputs.BcMin = 40.  # Minimum channel width
inputs.W = .07 # Narrowing constant--used to calibrate narrowing function.  
    # As a starting guide, estimate the percentage of bar that is vegetated
    # annually and double it.
inputs.alphatau = 32. # Shear stress threshold for channel narrowing

### Bank Erosion Terms

In [33]:
inputs.ErodeT = .55 # Fraction of near-bank sediment sourced from the active layer
inputs.MobilityThreshold = 10**-7 # Mobility threshold for initiation of bank erosion

### Avulsion Terms
Setting the avulsion threshold negative turns off avulsions. Use these with caution.

In [34]:
inputs.AvulsionExchange = .1
inputs.AvulsionThreshold = -1.25 # Minimum bank height below which avulsion will occur

## X. Reservoir Thickness and Exchange Parameters

### Sediment Reservoir Characteristics
These define the geometry of the sediment reservoir layers and the number of substrate layers.  Porosity of the deposit is also specified.

In [41]:
inputs.FloodplainL = 1.75 # Initial thickness of the active floodplain (m)
inputs.ActiveLayerL = .4 # Initial thickness of the Active Layer
inputs.LayerL = 1.5 # Thickness of substrate layers (m)
inputs.NLayers = 2 # Number of substrate layers
inputs.Hpb = 1.7 # Thickness of the point bar (constant through time)
inputs.lambdap = 0.5 # Porosity

### Reservoir exchange parameters
These parameters influece the rate at which sediment is exchanged from reservoir to reservoir.

In [42]:
inputs.Kbar = 1/1000000. # Parameter controlling fraction washload in point bar deposits
inputs.AlphaBed = .9 # Proportion of new substrate composed of active layer material
    # (verses load material)
inputs.AlphaBar = 1. # Parameter controlling similarity between bed material load 
    # and bar deposition
inputs.AlphaPartlyAlluvial = 0.9 # Parameter controlling similarity between bed
    # material load and deposition in the active layer of a partly alluvial node

## XI. TRACER PROPERTIES 

The tracer component has not been tested in this version of MAST-1D, but should 
theoretically work.  They may need to be initialized in order to compute a run.  The decaying tracer is set up as Cosmogenic 14C.

In [43]:
inputs.NTracers = 1 # Number of tracers
inputs.coj = [82.96, 1.98, 15.06] # production from different processes (at surface, presumably--not integrated over depth)
inputs.Lcj = [160., 738., 2688.] # Attenuation rates (g/cm^2)
inputs.Name = "'14C'"  # Name of radioisotopic tracer
inputs.DecayConst = 0.000121 # Decay constant (can be zero for a cnservative tracer)
inputs.ProductionRate = 15.1 # Nuclide production rate
inputs.FalloutRate = 0. # Nuclide fallout rate--most relevant for fine sediment.

inputs.TracerICFloodplain = 1.
inputs.TracerICActiveLayer = 0.
inputs.TracerICSubstrate = 0.
inputs.TracerBCFeed = 0.

## XII. OUTPUT SPECIFICATIONS 

There are three types of output:
A. Text files of node attributes for each node over time periods of equal intervals
    (for hydrographs and duration curves)
B. Text files of node attributes for each node on user-specified dates (currently 
    for hydrographs only)
C. JSON files with daily output at a given location. User specifies which 
    nodes and variables to write out. For hydrograph runs only.  



### Specify Output Folder

In [49]:
RunName = "ElwhaHydrographDemo_Simple_short" # Name of the run (a folder of outputs files will be created
    # under this name).
inputs.Outputfolder = os.path.join("Output","Demo",RunName) # Parent folder for 
    # the output folder 

### A.  General output

Any node property can be saved at regular intervals. These are saved for each node in the reach. The properties to export are specified as a list of strings defining the property. The string follows the same syntax one would use to access the note property. For instance, the to output node.Slope for all nodes at regular intervals, the string 'Slope' should be included in the .Outputvars list. For node properties that are lists, the index of the list must be specified.

For runs that are driven by a daily hydrograph, output can be provided very day. To limit the size of output files, this type of output is not saved for every node in the reach. As with varialbes that are output at regular intervals, the properties to save are listed as strings, but here they are strings of reach properties.  For instance, to save D50 of the grain size distribution of the active layer at the node with index 10, the string 'Node[10].ActiveLayer.GSD.D50' would be specified in DailyOutputvars. 

In [45]:
# Number of dataslices to save.  Will be written at equal time intervals.
inputs.NumberOfPrintouts = 50 
 
# List variables to save at regular intervals (strings of clsNode attributes)
inputs.Outputvars = ['Slope',
                     'Bf',
                     'Bc',
                     'DC.Qwf[-1]',
                     'DC.Sf[0]',
                     'DC.Hc[0]',
                     'DC.Hc[-1]',
                     'DC.Uc[-1]',
                     'DC.Uc[0]',
                     'DC.WSE[0]',
                     'DC.WSE[-1]',
                     'DC.Qw[0]',
                     'DC.Uc[0]',
                     'ActiveLayer.GSD.D50',
                     'ActiveLayer.GSD.D84',
                     'Load.QsavBedTot',
                     'Load.QsavBedTotFeed',
                     'Floodplain.GSD.D50',
                     'etabav',
                     'Substrate[-1].C.GSD.D50',
                     'CumulativeBedChange',
                     'Floodplain.L',
                     'Floodplain.GSD.F[0]',
                     'Floodplain.GSD.D84',
                     'CumulativeTotBedMaterialFeed',
                     'SLatSourceAv[-1]',
                     'CobbleMobility',
                     'WidenRate',
                     'NarrowRate',
                     'PointBarSubsurfaceGSD.D50',
                     ]

# List variables for daily output (strings of clsReach attributes)    
inputs.DailyOutputVars=['Node[0].DC.Qw[0]',
                        'Node[-1].DC.Qw[0]',
                        'Node[10].Load.QsjTot[0]',
                        'Node[0].CumulativeTotFeed',
                        'Node[-1].CumulativeTotFeed',
                        'Node[10].Load.QsAvkLoad[0]',
                        'Node[10].Load.QsAvkLoad[7]',
                        'Node[10].ActiveLayer.GSD.D50',
                        'Node[10].ActiveLayer.ExSed.InWidthChange[1]',
                        'Node[10].ActiveLayer.ExSed.InWidthChange[7]',
                        'Node[10].Bc',
                        'Node[10].NarrowRate',
                        'Node[10].WidenRate',
                        'Node[10].CumulativeNarrowing',
                        'Node[10].CumulativeWidening', 
                        'Node[10].ActiveLayer.GSD.F[0]',
                        'Node[10].ActiveLayer.GSD.F[1]',
                        'Node[10].ActiveLayer.GSD.F[7]',
                        'Node[10].ActiveLayer.GSD.F[-1]',
                        'Node[9].ActiveLayer.T[1,0]',
                        'Node[10].ActiveLayer.T[1,0]',
                        'Node[-1].ActiveLayer.T[1,0]',
                        'Node[-1].ActiveLayer.T[7,0]',
                        'Node[10].ActiveLayer.T[7,0]',
                        'Node[0].Floodplain.T[1,0]',
                        'Node[0].Load.TBedFeedk[0,0]',
                        'Node[1].Load.TBedFeedk[0,0]',
                        'Node[1].ActiveLayer.T[0,0]',
                        'Node[0].ActiveLayer.T[0,0]',
                        'Node[0].ActiveLayer.T[1,0]',
                        'Node[10].FkPointBarDepositAnnualAverage[1]',
                        'Node[10].Dfav[0]',
                        'Node[10].Dfav[1]'
                        ]

### Output on specific dates (for hydrograph runs)

In [46]:
# Dates (yyyy, m, dd) in which to output variables for model validation 
inputs.ValidateDates = [(2011, 9, 30),(2012, 9, 30),(2013, 9, 30),(2014, 9, 30),(2015, 9, 30),(2016, 9, 30)]  

# Variables (attributes of clsNode) in which to output on specific dates for model validation
inputs.Validatevars = ['CumulativeTotFeed','Bc','CumulativeNarrowing','CumulativeWidening','CumulativeTotalAvulsionWidth']

### Output daily (for hydrograph runs)
This feature has been superceded by the daily output variable method described earlier.  

Here, the nodes for which output will occur on a daily basis are specified.  The variables to save are hard-coded in clsOutputSpecs (which is why the method is deprecated). Note that the file sizes of daily output can be up to several megabytes, so use judiciosly.  It is better to use inputs.DailyOutputVars to specify daily output.

In [47]:
inputs.DailyNodes = [0,35] 

## Load the Hydrographs
The user does not modify this section.

### Define a function to load the hydrographs in case where there are several 

In [52]:
def load_hydrographs(inputs):
    #Initialize lists
    inputs.Qlist = []
    inputs.Qw = []
    inputs.p = []
    
    for f in inputs.DischargeFiles:
        # Load discharge files as a lists
        DischargeFile = os.path.join(os.pardir,"Discharge_Files", f)
        print(DischargeFile)
        Qlist = open(DischargeFile).readlines()
        Qlist = list(map(lambda x: float(x), Qlist))
        binsize = (max(Qlist)-min(Qlist))/inputs.NumberofHydroBins
        print('binsize = %s' %(binsize))
        # Create a duration curve from the list for setting up equilibrium floodplain
        # conditions and feed.  Other parameters can be customized (see ExtractDC function).
        Q = clsTimeSeries([],Qlist) 
        Qw, p = Q.CreateDurationCurve(binsize)
        print('number of bins made = %s' % len(Qw))
        inputs.Qlist.append(Qlist)
        inputs.Qw.append(Qw)
        inputs.p.append(p)
      
    return inputs

## RUN THE MODEL (Reading in any necessary hydrographs)
User does not modify.

In [58]:
    #  Checks to see if the specified output folder exists and creates it if it doesn't  
    directory = str(os.path.join(os.pardir,inputs.Outputfolder))
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Creates a duration curve for the discharge record for a hydrograph run and
    # creates a model object with the proper inputs.
    run = ""
    if inputs.Hydrograph == True:
        newinputs = load_hydrographs(inputs)
        run = clsModel(newinputs)
    else:
        run = clsModel(inputs)
        
    # Starts the model
    run.RunModel()

..\Discharge_Files\Qfile15Sep2011pres
binsize = 25.468171824133332
number of bins made = 15
Model setup complete!  Starting timesteps...
Saving regular output at count = 0
Saving validation output at specified date of: 2011-09-30 
2011-10-01
Saving regular output at count = 37
2011-11-01
dt in model set to high flow dt of 10800.0
dt in model set to high flow dt of 10800.0
dt in model set to high flow dt of 10800.0
Saving regular output at count = 75
2011-12-01
dt in model set to high flow dt of 10800.0
2012-01-01
dt in model set to high flow dt of 10800.0
Saving regular output at count = 113
2012-02-01
Saving regular output at count = 151
2012-03-01
Saving regular output at count = 188
2012-04-01
dt in model set to high flow dt of 10800.0
Saving regular output at count = 226
2012-05-01
2012-06-01
Saving regular output at count = 264
2012-07-01
Saving regular output at count = 302
2012-08-01
Saving regular output at count = 339
2012-09-01
Saving regular output at count = 377
Saving vali

### Start the animation tool

In [68]:
%run Animator.py

This should start a widget that allows any of the attributes that were output in step XII.A to be animated on the screen. To use the animator tool, the user must navigate to the output folder that was specified at the beginning of step XII.

### Start the time series graphing tool

In [69]:
%run ATimeseriesPlotter.py

This should start a widget that allows up to three timeseries withing a given node to be plotted on a three-part figure. The locations and variables where output is available were specified in step XII.A. As with the animator tool, the user must navigate to the output folder that was specified at the beginning of step XII.